In [1]:
import pandas as pd
from scipy.special import binom
from scipy.misc import factorial
from math import log

In [2]:
data = pd.read_csv('autoclaved-1529600920.coincidence-stat', sep='\t', names=['rep', 'msm'])

In [3]:
data.head()

,rep,msm
0,2,1021
1,2,1023
2,2,999
3,4,1002
4,4,1002


In [4]:
print 'Number of colliding timestamps:', len(data.rep)
print 'Number of reports:', data.rep.sum()
print 'Number of measurements:', data.msm.sum()

Number of colliding timestamps: 316084
Number of reports: 753235
Number of measurements: 40440216


In [5]:
def prob(bits):
    space = (2.**bits)**data.rep
    k = data.rep
    n = (2.**bits) - data.msm
    good = binom(n + k , k) * factorial(data.rep)
    perexp = good / space
    return perexp

In [6]:
for i in [32, 31, 30, 29, 28, 27, 26, 25, 24, 23, 22, 21, 20]:
    perexp = prob(i)
    p = perexp.prod() # all at once
    print i, '%6.1f' % (-log(p)/log(2)), p

32    0.0 0.968998231107
31    0.1 0.938957560662
30    0.2 0.881641258555
29    0.4 0.777291160072
28    0.7 0.604181085148
27    1.5 0.365033666199
26    2.9 0.133247945814
25    5.8 0.0177541454083
24   11.6 0.000315147922694
23   23.3 9.92403912673e-08
22   46.5 9.81781524666e-15
21   93.1 9.51871872619e-29
20  186.2 8.6164609409e-57


In [7]:
# per-experiment probability of success is still quite high, so some set of keys can be bruteforced for 20bit tail
pd.Series(perexp).describe()

count    316084.000000
mean          0.999593
std           0.001830
min           0.906941
25%           0.999957
50%           0.999999
75%           0.999999
max           1.000020
dtype: float64

In [8]:
step = 1234
print 'Number of buckets', len(perexp) / step
pd.Series([perexp[i:i+step].product() for i in range(0, len(perexp), step)]).describe()

Number of buckets 256


count    257.000000
mean       0.668918
std        0.220726
min        0.002041
25%        0.533609
50%        0.687229
75%        0.842643
max        0.989087
dtype: float64